In [1]:
%pip install numpy==1.25.2 pandas==1.5.3 matplotlib==3.7.1 seaborn==0.13.1 scikit-learn==1.2.2 sklearn-pandas==2.2.0 -q --user
%pip install opendatasets pandas numpy scikit-learn xgboost lightgbm tabulate category_encoders



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
#import some necessary librairies
import datetime
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


from scipy import stats
from scipy.stats import norm, skew #for some statistics


pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points


from subprocess import check_output
#print(check_output(["dir", "C:/Users/anirb/aiml/UT-Austin-PGP/input/"]).decode("utf8")) #check the files available in the directory

# Loading the Training Data and using the Training Data to build the Model

## Data Loading

### Load Training

In [3]:
#Now let's import and put the train and test datasets in  pandas dataframe
folderPath = 'C:/Users/anirb/aiml/UT-Austin-PGP'
train = pd.read_csv(folderPath + '/input/train.csv')
test = pd.read_csv(folderPath + '/input/test.csv')


In [4]:
##display the first five rows of the test dataset.
test.head(5)


,Registration Number,Cuisine,City,Restaurant Location,Opening Day of Restaurant,Facebook Popularity Quotient,Endorsed By,Instagram Popularity Quotient,Fire Audit,Liquor License Obtained,...,Overall Restaurant Rating,Live Music Rating,Comedy Gigs Rating,Value Deals Rating,Live Sports Rating,Ambience,Lively,Service,Comfortablility,Privacy
0,20001,"tibetan,italian",Bangalore,Near Business Hub,13-07-2010,78.000,Tier A Celebrity,69.000,1,1,...,6.000,NaN,2.000,NaN,NaN,5,2,2,6,0
1,20002,"tibetan,italian",Hyderabad,Near Party Hub,05-09-2011,89.170,Not Specific,96.000,1,1,...,9.000,3.000,NaN,NaN,6.000,7,8,5,1,8
2,20003,"algerian,belgian",Hyderabad,Near Party Hub,12-04-2011,84.000,Not Specific,86.000,1,1,...,6.000,3.000,NaN,NaN,NaN,9,7,5,1,5
3,20004,"tibetan,greek",-1,Near Party Hub,16-01-2005,79.380,Not Specific,74.400,0,1,...,9.000,6.000,NaN,NaN,NaN,6,7,6,4,7
4,20005,"cuban,british",Pune,Near Party Hub,10-11-2008,84.670,Not Specific,86.460,1,1,...,NaN,NaN,NaN,NaN,NaN,4,4,7,7,3


In [5]:
#check the numbers of samples and features
print("The train data size before dropping Id feature is : {} ".format(train.shape))
print("The test data size before dropping Id feature is : {} ".format(test.shape))

#Save the 'Id' column
# train_ID = train['Registration Number']
# test_ID = test[]

#Now drop the  'Id' colum since it's unnecessary for  the prediction process.
# train.drop("Registration Number", axis = 1, inplace = True)
# test.drop("Registration Number", axis = 1, inplace = True)

#check again the data size after dropping the 'Id' variable
print("\nThe train data size after dropping Id feature is : {} ".format(train.shape)) 
print("The test data size after dropping Id feature is : {} ".format(test.shape))
 


The train data size before dropping Id feature is : (3493, 34) 
The test data size before dropping Id feature is : (500, 33) 

The train data size after dropping Id feature is : (3493, 34) 
The test data size after dropping Id feature is : (500, 33) 


NameError: name 'test1' is not defined

## Data Cleanup

### Target Variables

In [ ]:
sns.distplot(train['Annual Turnover'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(train['Annual Turnover'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('Annual Turnover distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(train['Annual Turnover'], plot=plt)
plt.show()

In [ ]:
#We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
train["Annual Turnover"] = np.log1p(train["Annual Turnover"])

#Check the new distribution 
sns.distplot(train['Annual Turnover'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(train['Annual Turnover'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('Annual Turnover distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(train['Annual Turnover'], plot=plt)
plt.show()


In [ ]:
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['Annual Turnover'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

### Fix City

In [ ]:
all_data['City'] = all_data['City'].replace('-1', 'NOCITY')
all_data['City'] = all_data['City'].str.upper().str.strip()

In [ ]:
replacement_pattern = r"^(BANGALORE|BANGLORE|BENGALURU|Asifabadbanglore|Banaglore)$"  # Consider using raw string for regex

# Replace matching values with 'BANGALORE' (case-sensitive)
all_data['City'] = all_data['City'].str.replace(replacement_pattern, 'BANGALORE')


replacement_pattern = r"^(BHUBANESHWAR|BHUBANESWAR|BHUBNESHWAR)$"  # Consider using raw string for regex
all_data['City'] = all_data['City'].str.replace(replacement_pattern, 'BHUBANESHWAR')


replacement_pattern = r"^(TRIVANDRUM|THIRUVANANTHAPURAM|THIRUVANANTHAPURAM)$"  # Consider using raw string for regex
all_data['City'] = all_data['City'].str.replace(replacement_pattern, 'TRIVANDRUM')

replacement_pattern = r"^(GURAGAON|GURGAON)$"  # Consider using raw string for regex
all_data['City'] = all_data['City'].str.replace(replacement_pattern, 'GURAGAON')

replacement_pattern = r"^(NEW DEHLI|NEW DELHI)$"  # Consider using raw string for regex
all_data['City'] = all_data['City'].str.replace(replacement_pattern, 'NEW DELHI')


replacement_pattern = r"^(VISAKHAPATNAM|VIZAG|VSAKHAPTTNAM)$"  # Consider using raw string for regex
all_data['City'] = all_data['City'].str.replace(replacement_pattern, 'VIZAG')

all_data.loc[all_data['City'].isin((all_data['City'].value_counts()[all_data['City'].value_counts() < 6]).index), 'City'] = 'OTHER'

all_data['City'].value_counts().sort_values(ascending=False)



In [ ]:
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head(20)

In [ ]:
f, ax = plt.subplots(figsize=(15, 12))
plt.xticks(rotation='vertical')
sns.barplot(x=all_data_na.index, y=all_data_na)
plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)
plt.title('Percent missing data by feature', fontsize=15)

In [ ]:
#Correlation map to see how features are correlated with SalePrice
corrmat = train.corr()
plt.subplots(figsize=(12,9))
sns.heatmap(corrmat, vmax=0.9, square=True,color='blue')

In [ ]:
all_data["Live Sports Rating"] = all_data["Live Sports Rating"].transform(lambda x: x.fillna(x.median()))
all_data["Live Sports Rating"].value_counts(dropna=False)

In [ ]:
all_data["Value Deals Rating"] = all_data["Value Deals Rating"].transform(lambda x: x.fillna(x.median()))
all_data['Value Deals Rating'].value_counts(dropna=False)


In [ ]:
all_data["Comedy Gigs Rating"] = all_data["Comedy Gigs Rating"].transform(lambda x: x.fillna(x.median()))
all_data['Comedy Gigs Rating'].value_counts(dropna=False)


In [ ]:
all_data["Live Music Rating"] = all_data["Live Music Rating"].transform(lambda x: x.fillna(x.median()))
all_data['Live Music Rating'].value_counts(dropna=False)


In [ ]:
all_data["Overall Restaurant Rating"] = all_data["Overall Restaurant Rating"].transform(lambda x: x.fillna(x.median()))
all_data['Overall Restaurant Rating'].value_counts(dropna=False)


In [ ]:
all_data["Facebook Popularity Quotient"] = all_data["Facebook Popularity Quotient"].transform(lambda x: x.fillna(x.mean()))
all_data['Facebook Popularity Quotient'].value_counts(dropna=False)


In [ ]:
all_data["Instagram Popularity Quotient"] = all_data["Instagram Popularity Quotient"].transform(lambda x: x.fillna(x.mean()))
all_data['Instagram Popularity Quotient'].value_counts(dropna=False)

In [ ]:
all_data["Ambience"] = all_data["Ambience"].transform(lambda x: x.fillna(x.median()))
all_data['Ambience'].value_counts(dropna=False)


In [ ]:
all_data["Resturant Tier"] = all_data["Resturant Tier"].transform(lambda x: x.fillna(x.median()))
all_data['Resturant Tier'].value_counts(dropna=False)


In [ ]:
#Check remaining missing values if any 
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()

### City TE

In [ ]:
columns_to_replace = [col for col in train.columns if col != 'Annual Turnover']
# Update train with corresponding columns from all_data (excluding Annual Turnover)
train.update(all_data[:ntrain][columns_to_replace])

#train = all_data[:ntrain]
import category_encoders as ce
encoder = ce.TargetEncoder()
train['City_TE'] = encoder.fit_transform(train['City'],np.log1p( train['Annual Turnover']))


# Assuming City_encoded in df_train has target means
df_target_encoding = train[['City_TE', 'City']]  # Optional: Select relevant columns
df_target_encoding = df_target_encoding.drop_duplicates()
# # Assuming City_target_encoding exists in df_train_without_duplicates
city_target_means = df_target_encoding.set_index('City')['City_TE'].to_dict()
print(city_target_means)

# # # Example usage on df_test (replace with your actual usage)
test  = all_data[ntrain:]

test['City_TE'] = test['City'].replace(city_target_means)
# # print(test['City_TE'])
all_data = pd.concat((train, test))
all_data.drop(['Annual Turnover'], axis=1, inplace=True)
#all_data.drop(['City'], axis=1, inplace=True)

#test
all_data

# print(train['City_TE'])
# df_test.head()



### Restaurant Theme TE

In [ ]:
columns_to_replace = [col for col in train.columns if col != 'Annual Turnover']
# Update train with corresponding columns from all_data (excluding Annual Turnover)
train.update(all_data[:ntrain][columns_to_replace])

import category_encoders as ce
encoder = ce.TargetEncoder()
train['Theme_TE'] = encoder.fit_transform(train['Restaurant Theme'],np.log1p( train['Annual Turnover']))


# Assuming City_encoded in df_train has target means
df_target_encoding = train[['Theme_TE', 'Restaurant Theme']]  # Optional: Select relevant columns
df_target_encoding = df_target_encoding.drop_duplicates()
# # Assuming City_target_encoding exists in df_train_without_duplicates
theme_target_means = df_target_encoding.set_index('Restaurant Theme')['Theme_TE'].to_dict()
# print(city_target_means)

# # Example usage on df_test (replace with your actual usage)
test  = all_data[ntrain:]
test['Theme_TE'] = test['Restaurant Theme'].replace(theme_target_means)
# print(test['City_TE'])
all_data = pd.concat((train, test))
all_data.drop(['Annual Turnover'], axis=1, inplace=True)

#all_data.drop('Restaurant Theme', axis=1, inplace=True)

all_data

### Cuisine Type TE

In [ ]:
# columns_to_replace = [col for col in train.columns if col != 'Annual Turnover']
# # Update train with corresponding columns from all_data (excluding Annual Turnover)
# train.update(all_data[:ntrain][columns_to_replace])

# import category_encoders as ce
# encoder = ce.TargetEncoder()
# train['Cuisine_TE'] = encoder.fit_transform(train['Cuisine'],np.log1p( train['Annual Turnover']))


# # Assuming City_encoded in df_train has target means
# df_target_encoding = train[['Cuisine_TE', 'Cuisine']]  # Optional: Select relevant columns
# df_target_encoding = df_target_encoding.drop_duplicates()
# # # Assuming City_target_encoding exists in df_train_without_duplicates
# cuisine_target_means = df_target_encoding.set_index('Cuisine')['Cuisine_TE'].to_dict()
# # print(city_target_means)

# # # Example usage on df_test (replace with your actual usage)
# test  = all_data[ntrain:]
# test['Cuisine_TE'] = test['Cuisine'].replace(cuisine_target_means)
# # print(test['City_TE'])
# all_data = pd.concat((train, test))
# all_data.drop(['Annual Turnover'], axis=1, inplace=True)

# #all_data.drop(['Cuisine'], axis=1, inplace=True)

# all_data

In [ ]:
#all_data = pd.concat([all_data.drop('Cuisine', 1), all_data['Cuisine'].str.get_dummies()], 1)

In [ ]:
# all_data = pd.concat([all_data.drop('Resturent Tier', 1), all_data['Resturent Tier'].str.get_dummies()], 1)

In [ ]:
#all_data = pd.concat([all_data.drop('Restaurant Location', 1), all_data['Restaurant Location'].str.get_dummies()], 1)

In [ ]:
all_data = pd.get_dummies(all_data,columns=["Restaurant Location"])

all_data = pd.get_dummies(all_data,columns=["Fire Audit"])
all_data = pd.get_dummies(all_data,columns=["Liquor License Obtained"])
all_data = pd.get_dummies(all_data,columns=["Situated in a Multi Complex"])
all_data = pd.get_dummies(all_data,columns=["Dedicated Parking"])

all_data = pd.get_dummies(all_data,columns=["Open Sitting Available"])
all_data = pd.get_dummies(all_data,columns=["Restaurant Zomato Rating"])
all_data = pd.get_dummies(all_data,columns=["Order Wait Time"])
all_data = pd.get_dummies(all_data,columns=["Restaurant City Tier"])
all_data = pd.get_dummies(all_data,columns=["Hygiene Rating"])
all_data = pd.get_dummies(all_data,columns=["Food Rating"])
all_data = pd.get_dummies(all_data,columns=["Overall Restaurant Rating"])

all_data = pd.get_dummies(all_data,columns=["Staff Responsivness"])
all_data = pd.get_dummies(all_data,columns=["Value for Money"])
all_data = pd.get_dummies(all_data,columns=["Live Music Rating"])
all_data = pd.get_dummies(all_data,columns=["Comedy Gigs Rating"])
all_data = pd.get_dummies(all_data,columns=["Value Deals Rating"])

all_data = pd.get_dummies(all_data,columns=["Live Sports Rating"])
all_data = pd.get_dummies(all_data,columns=["Ambience"])
all_data = pd.get_dummies(all_data,columns=["Lively"])
all_data = pd.get_dummies(all_data,columns=["Service"])
all_data = pd.get_dummies(all_data,columns=["Comfortablility"])
all_data = pd.get_dummies(all_data,columns=["Privacy"])

all_data = pd.get_dummies(all_data,columns=["Restaurant Type"])

all_data = pd.get_dummies(all_data,columns=["Cuisine"])
all_data = pd.get_dummies(all_data,columns=["Endorsed By"])






# 'Restaurant Type', 'Staff Responsivness',
#        'Value for Money', 'Live Music Rating', 'Comedy Gigs Rating',
#        'Value Deals Rating', 'Live Sports Rating', 'Ambience', 'Lively',
#        'Service', 'Comfortablility', 'Privacy'


In [ ]:
all_data.columns

In [ ]:
# columnsToOHE = ['Cuisine','Restaurant Location',
#        'Endorsed By',  'Fire Audit',
#        'Liquor License Obtained', 'Situated in a Multi Complex',
#        'Dedicated Parking', 'Open Sitting Available', 
#        'Restaurant Type',  'Restaurant Zomato Rating',
#        'Restaurant City Tier', 'Order Wait Time', 'Staff Responsivness',
#        'Value for Money', 'Hygiene Rating', 'Food Rating',
#        'Overall Restaurant Rating', 'Live Music Rating', 'Comedy Gigs Rating',
#        'Value Deals Rating', 'Live Sports Rating', 'Ambience', 'Lively',
#        'Service', 'Comfortablility', 'Privacy']

# # prefixes = ["Cuisine",'Location',
# #        'Endorsed',  'Fire',
# #        'Liquor', 'MultiComplex',
# #        'Parking', 'OpenSitting', 
# #        'RestaurantType',  'RestaurantZomato',
# #        'CityTier', 'OrderWaitTime', 'Staff',
# #        'ValueMoney', 'HygieneRating', 'FoodRating',
# #        'OverallRating', 'LiveMusic', 'ComedyGigs',
# #        'ValueDeals', 'LiveSports', 'Ambience', 'Lively',
# #        'Service', 'Comfortablility', 'Privacy']

# # for col in columnsToOHE:
# #   all_data = pd.get_dummies(all_data, columns=[col])  # Encode each column separately

# # pd.get_dummies(all_data , columns=columnsToOHE)
# #all_data = pd.get_dummies(all_data,  columns=['Cuisine'])
# # pd.get_dummies(all_data, prefix='Fire' , columns=['Fire Audit'])
# # pd.get_dummies(all_data, prefix='Fire' , columns=['Fire Audit'])
# # pd.get_dummies(all_data, prefix='Fire' , columns=['Fire Audit'])
# # pd.get_dummies(all_data, prefix='Fire' , columns=['Fire Audit'])
# # pd.get_dummies(all_data, prefix='Fire' , columns=['Fire Audit'])
# # pd.get_dummies(all_data, prefix='Fire' , columns=['Fire Audit'])
# # pd.get_dummies(all_data, prefix='Fire' , columns=['Fire Audit'])
# # pd.get_dummies(all_data, prefix='Fire' , columns=['Fire Audit'])
# # pd.get_dummies(all_data, prefix='Fire' , columns=['Fire Audit'])
# # pd.get_dummies(all_data, prefix='Fire' , columns=['Fire Audit'])
# # pd.get_dummies(all_data, prefix='Fire' , columns=['Fire Audit'])
# # pd.get_dummies(all_data, prefix='Fire' , columns=['Fire Audit'])
# # pd.get_dummies(all_data, prefix='Fire' , columns=['Fire Audit'])

# #all_data = pd.concat([all_data.drop('Fire Audit', 1), all_data['Fire Audit'].str.get_dummies()], 1)
# # all_data = pd.concat([all_data.drop('Liquor License Obtained', 1), all_data['Liquor License Obtained'].str.get_dummies()], 1)
# # all_data = pd.concat([all_data.drop('Situated in a Multi Complex', 1), all_data['Situated in a Multi Complex'].str.get_dummies()], 1)
# # all_data = pd.concat([all_data.drop('Open Sitting Available', 1), all_data['Open Sitting Available'].str.get_dummies()], 1)
# # all_data = pd.concat([all_data.drop('Order Wait Time', 1), all_data['Order Wait Time'].str.get_dummies()], 1)
# # all_data = pd.concat([all_data.drop('Staff Responsivness', 1), all_data['Staff Responsivness'].str.get_dummies()], 1)
# # all_data = pd.concat([all_data.drop('Value for Money', 1), all_data['Value for Money'].str.get_dummies()], 1)
# # all_data = pd.concat([all_data.drop('Hygiene Rating', 1), all_data['Hygiene Rating'].str.get_dummies()], 1)
# # all_data = pd.concat([all_data.drop('Food Rating', 1), all_data['Food Rating'].str.get_dummies()], 1)
# # all_data = pd.concat([all_data.drop('Live Music Rating', 1), all_data['Live Music Rating'].str.get_dummies()], 1)
# # all_data = pd.concat([all_data.drop('Comedy Gigs Rating', 1), all_data['Comedy Gigs Rating'].str.get_dummies()], 1)
# # all_data = pd.concat([all_data.drop('Value Deals Rating', 1), all_data['Value Deals Rating'].str.get_dummies()], 1)
# # all_data = pd.concat([all_data.drop('Ambience', 1), all_data['Ambience'].str.get_dummies()], 1)
# # all_data = pd.concat([all_data.drop('Lively', 1), all_data['Lively'].str.get_dummies()], 1)
# # all_data = pd.concat([all_data.drop('Service', 1), all_data['Service'].str.get_dummies()], 1)
# # all_data = pd.concat([all_data.drop('Comfortablility', 1), all_data['Comfortablility'].str.get_dummies()], 1)
# # all_data = pd.concat([all_data.drop('Privacy', 1), all_data['Privacy'].str.get_dummies()], 1)
# all_data.columns()


In [ ]:
# all_data = pd.concat([all_data.drop('Endorsed By', 1), all_data['Endorsed By'].str.get_dummies()], 1)


In [ ]:
# all_data = pd.concat([all_data.drop('Restaurant Type', 1), all_data['Restaurant Type'].str.get_dummies()], 1)
# all_data

### Date Field

In [ ]:
#all_data['Opening Day of Restaurant'] =  pd.to_datetime(all_data['Opening Day of Restaurant'], format='%d-%m-%Y')
#print(all_data['Opening Day of Restaurant'].dtype)
launch_date = datetime.datetime(2015, 3, 23)
# scale days open
all_data['Days Open'] = (launch_date - pd.to_datetime(all_data['Opening Day of Restaurant'], format='%d-%m-%Y')).dt.days / 1000
all_data['Opening Year'] =  pd.to_datetime(all_data['Opening Day of Restaurant'], format='%d-%m-%Y').dt.year
all_data['Opening Month'] =  pd.to_datetime(all_data['Opening Day of Restaurant'], format='%d-%m-%Y').dt.month
#all_data['Opening Year'].astype(str)
all_data['Opening Month'].value_counts()

all_data.drop('Opening Day of Restaurant', axis=1, inplace=True)
all_data


#### Year Month OHE

In [ ]:
all_data = pd.get_dummies(all_data,columns=["Opening Year"])
all_data = pd.get_dummies(all_data,columns=["Opening Month"])

all_data


In [ ]:
all_data.drop(['City'], axis=1, inplace=True)
# all_data.drop(['Cuisine'], axis=1, inplace=True)
all_data.drop(['Restaurant Theme'], axis=1, inplace=True)
all_data

In [ ]:
df_train = all_data[:ntrain]
df_test = all_data[ntrain:]
df_train = pd.concat([y_train['Annual Turnover'],df_train], axis=1)
df_test

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# plt.figure(figsize=(15, 7))
# sns.heatmap(df_train.corr(), annot=True, vmin=-1, vmax=1, fmt=".2f", cmap="Spectral") # Complete the code to get the heatmap of the data
# plt.show()

## Split Train and Test

In [ ]:
from sklearn.model_selection import train_test_split


train_df, val_df = train_test_split(df_train, test_size=0.2, random_state=42)

len(train_df), len(val_df) , len(df_test)

## Train and Validation Dataset

In [ ]:
train_targets = train_df['Annual Turnover']
# Column to exclude
columns_to_exclude = ['Annual Turnover','Registration Number']
# Select all columns except 'col2' (using drop with subset and inplace=False)

train_inputs  = train_df.drop(columns_to_exclude, axis=1, inplace=False)  # Keep original intact
val_inputs = val_df.drop(columns_to_exclude, axis=1, inplace=False)  
val_targets = val_df['Annual Turnover']

### Scale

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

numeric = ['Facebook Popularity Quotient', 'Instagram Popularity Quotient',
      'Days Open'
]

# Fit the MinMaxScaler to the training data
scaler.fit(train_inputs)

# Use the scaler to transform the training and test sets
train_inputs[numeric] = scaler.fit_transform(train_inputs[numeric])
val_inputs[numeric] = scaler.fit_transform(val_inputs[numeric])
# X_test_scaled = scaler.transform(val_inputs)
df_test[numeric]= scaler.fit_transform(df_test[numeric])

### PCA

# Try Different Models

#### Change Column Names

In [ ]:
import re
train_inputs = train_inputs.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
val_inputs = val_inputs.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df_test = df_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
linreg_model = LinearRegression()
linreg_model.fit(train_inputs, train_targets)
LinearRegression()


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
train_preds = linreg_model.predict(train_inputs)
val_preds = linreg_model.predict(val_inputs)

#row_61946 = val_inputs.loc[df_train['Registration Number'] == 61946]
#print(row_61946)
#val_preds_RN = linreg_model.predict(row_61946)
#validation_mse_RN = np.sqrt(mean_squared_error([22000000], val_preds_RN))
#print(validation_mse_RN)

In [ ]:
train_rmse_linreg = np.sqrt(mean_squared_error(train_preds, train_targets))
test_rmse_linreg = np.sqrt(mean_squared_error(val_preds, val_targets))
print(f'Train RMSE Linear Reg: {train_rmse_linreg:n}')
print(f'Test RMSE Linear Reg: {test_rmse_linreg:n}')
import matplotlib.pyplot as plt


## Ridge Linear Regression


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso


params_ridge = {
    'alpha' : [.01, .1, .5, .7, .9, .95, .99, 1, 5, 10, 20],
    'fit_intercept' : [True, False],
#    'normalize' : [True,False],
    'solver' : ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
}

ridge_model = Ridge()
ridge_regressor = GridSearchCV(ridge_model, params_ridge, scoring='neg_root_mean_squared_error', cv=5, n_jobs=-1)
ridge_regressor.fit(train_inputs, train_targets)
# print(f'Optimal alpha: {ridge_regressor.best_params_["alpha"]:.2f}')
# print(f'Optimal fit_intercept: {ridge_regressor.best_params_["fit_intercept"]}')
# #print(f'Optimal normalize: {ridge_regressor.best_params_["normalize"]}')
# print(f'Optimal solver: {ridge_regressor.best_params_["solver"]}')
# print(f'Best score: {ridge_regressor.best_score_}')

In [ ]:
ridge_model = Ridge(alpha=ridge_regressor.best_params_["alpha"], fit_intercept=ridge_regressor.best_params_["fit_intercept"], 
                    solver=ridge_regressor.best_params_["solver"])
ridge_model.fit(train_inputs, train_targets)
train_preds = ridge_model.predict(train_inputs)
val_preds = ridge_model.predict(val_inputs)
print('Train r2 score: ', r2_score(train_preds, train_targets))
print('Test r2 score: ', r2_score(val_targets, val_preds))
train_rmse_ridge = np.sqrt(mean_squared_error(train_preds, train_targets))
test_rmse_ridge = np.sqrt(mean_squared_error(val_preds, val_targets))
print(f'Train RMSE Ridge: {train_rmse_ridge:n}')
print(f'Test RMSE Ridge: {test_rmse_ridge:n}')

In [ ]:
# Lasso Model Feature Importance
lasso_feature_coef = pd.Series(index = train_inputs.columns, data = np.abs(ridge_model.coef_))
lasso_feature_coef.sort_values().plot(kind = 'bar', figsize = (30,5));

## Lasso Model

In [ ]:
params_lasso = {
    'alpha' : [.01, .1, .5, .7, .9, .95, .99, 1, 5, 10, 20],
    'fit_intercept' : [True, False],
}

lasso_model = Lasso()
lasso_regressor = GridSearchCV(lasso_model, params_lasso, scoring='neg_root_mean_squared_error', cv=5, n_jobs=-1)
lasso_regressor.fit(train_inputs, train_targets)
print(f'Optimal alpha: {lasso_regressor.best_params_["alpha"]:.2f}')
print(f'Optimal fit_intercept: {lasso_regressor.best_params_["fit_intercept"]}')
print(f'Best score: {lasso_regressor.best_score_}')

In [ ]:
lasso_model = Lasso(alpha=lasso_regressor.best_params_["alpha"], fit_intercept=lasso_regressor.best_params_["fit_intercept"], 
                    )
lasso_model.fit(train_inputs, train_targets)
train_preds = lasso_model.predict(train_inputs)
val_preds = lasso_model.predict(val_inputs)
print('Train r2 score: ', r2_score(train_preds, train_targets))
print('Test r2 score: ', r2_score(val_targets, val_preds))
train_rmse_lasso = np.sqrt(mean_squared_error(train_preds, train_targets))
test_rmse_lasso = np.sqrt(mean_squared_error(val_targets, val_preds))
print(f'Train RMSE: {train_rmse_lasso:.4f}')
print(f'Test RMSE: {test_rmse_lasso:.4f}')

In [ ]:
# Lasso Model Feature Importance
lasso_feature_coef = pd.Series(index = train_inputs.columns, data = np.abs(lasso_model.coef_))
lasso_feature_coef.sort_values().plot(kind = 'bar', figsize = (30,5));

## XG Boost Model

In [ ]:
from xgboost import XGBRegressor, plot_importance
# Scale the data using StandardScaler
from sklearn.preprocessing import StandardScaler


params_xgb = {
    'learning_rate': [.1, .5, .7, .9, .95, .99, 1],
    'colsample_bytree': [.3, .4, .5, .6],
    'max_depth': [4],
    'alpha': [3],
    'subsample': [.5],
    'n_estimators': [30, 70, 100, 200]
}

xgb_model = XGBRegressor()
xgb_regressor = GridSearchCV(xgb_model, params_xgb, scoring='neg_root_mean_squared_error', cv = 10, n_jobs = -1)
xgb_regressor.fit(train_inputs, train_targets)
print(f'Optimal lr: {xgb_regressor.best_params_["learning_rate"]}')
print(f'Optimal colsample_bytree: {xgb_regressor.best_params_["colsample_bytree"]}')
print(f'Optimal n_estimators: {xgb_regressor.best_params_["n_estimators"]}')
print(f'Best score: {xgb_regressor.best_score_}')

In [ ]:
xgb_model = XGBRegressor(learning_rate=xgb_regressor.best_params_["learning_rate"], 
                         colsample_bytree=xgb_regressor.best_params_["colsample_bytree"], 
                         max_depth=4, alpha=3, subsample=.5, 
                         n_estimators=xgb_regressor.best_params_["n_estimators"], n_jobs=-1)
xgb_model.fit(train_inputs, train_targets)

train_preds = xgb_model.predict(train_inputs)
val_preds = xgb_model.predict(val_inputs)
print('Train r2 score: ', r2_score(train_preds, train_targets))
print('Test r2 score: ', r2_score(val_targets, val_preds))
train_rmse_xgb = np.sqrt(mean_squared_error(train_preds, train_targets))
test_rmse_xgb = np.sqrt(mean_squared_error(val_targets, val_preds))
print(f'Train RMSE XGB: {train_rmse_xgb:n}')
print(f'Test RMSE XGB: {test_rmse_xgb:n}')

In [ ]:
# XG Forest Model Feature Importance
rf_feature_importance = pd.Series(index = train_inputs.columns, data = np.abs(xgb_model.feature_importances_))
n_features = (rf_feature_importance>0).sum()
print(f'{n_features} features with reduction of {(1-n_features/len(rf_feature_importance))*100:2.2f}%')
rf_feature_importance.sort_values().plot(kind = 'bar', figsize = (30,5));

## Random Forest Regressor

### Grid Search

In [ ]:
# from sklearn.ensemble import RandomForestRegressor

# params_rf = {
#     'max_depth': [10, 30, 35, 50, 65, 75, 100],
#     'max_features': [.3, .4, .5, .6],
#     'min_samples_leaf': [3, 4, 5],
#     'min_samples_split': [8, 10, 12],
#     'n_estimators': [30, 50, 100, 200]
# }

# rf = RandomForestRegressor()
# rf_regressor = GridSearchCV(rf, params_rf, scoring='neg_root_mean_squared_error', cv = 10, n_jobs = -1)
# rf_regressor.fit(train_inputs, train_targets)
# print(f'Optimal depth: {rf_regressor.best_params_["max_depth"]}')
# print(f'Optimal max_features: {rf_regressor.best_params_["max_features"]}')
# print(f'Optimal min_sample_leaf: {rf_regressor.best_params_["min_samples_leaf"]}')
# print(f'Optimal min_samples_split: {rf_regressor.best_params_["min_samples_split"]}')
# print(f'Optimal n_estimators: {rf_regressor.best_params_["n_estimators"]}')
# print(f'Best score: {rf_regressor.best_score_}')

In [ ]:
# rf_model = RandomForestRegressor(max_depth=rf_regressor.best_params_["max_depth"], 
#                                  max_features=rf_regressor.best_params_["max_features"], 
#                                  min_samples_leaf=rf_regressor.best_params_["min_samples_leaf"], 
#                                  min_samples_split=rf_regressor.best_params_["min_samples_split"], 
#                                  n_estimators=rf_regressor.best_params_["n_estimators"], 
#                                  n_jobs=-1, oob_score=True)
# rf_model.fit(train_inputs, train_targets)
# train_preds = rf_model.predict(train_inputs)
# val_preds = rf_model.predict(val_inputs)
# print('Train r2 score: ', r2_score(train_preds, train_targets))
# print('Test r2 score: ', r2_score(val_preds,val_targets ))
# train_rmse = np.sqrt(mean_squared_error(train_preds, train_targets))
# test_rmse = np.sqrt(mean_squared_error(val_targets, val_preds))
# print(f'Train RMSE: {train_rmse:n}')
# print(f'Test RMSE: {test_rmse:n}')

In [ ]:
# # Random Forest Model Feature Importance
# rf_feature_importance = pd.Series(index = train_inputs.columns, data = np.abs(rf_model.feature_importances_))
# n_features = (rf_feature_importance>0).sum()
# print(f'{n_features} features with reduction of {(1-n_features/len(rf_feature_importance))*100:2.2f}%')
# rf_feature_importance.sort_values().plot(kind = 'bar', figsize = (13,5));

### With RandomizedSearch instead of GridSearchCV

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import uniform, randint
from sklearn.model_selection import RandomizedSearchCV

params = {
    "n_estimators": randint(10,1000),
    "max_depth": randint(1,10),
    "min_samples_split": uniform(0.1,0.8),
    'max_features':['auto', 'sqrt', 'log2']
}

rf = RandomForestRegressor()
rf_regressor = RandomizedSearchCV(rf, params, scoring='neg_root_mean_squared_error', n_iter=100, cv = 6,  return_train_score=True, verbose=3, n_jobs=-1)
rf_regressor.fit(train_inputs, train_targets)
# print(f'Optimal depth: {rf_regressor.best_params_["max_depth"]}')
# print(f'Optimal max_features: {rf_regressor.best_params_["max_features"]}')
# print(f'Optimal min_sample_leaf: {rf_regressor.best_params_["min_samples_leaf"]}')
# print(f'Optimal min_samples_split: {rf_regressor.best_params_["min_samples_split"]}')
# print(f'Optimal n_estimators: {rf_regressor.best_params_["n_estimators"]}')
# print(f'Best score: {rf_regressor.best_score_}')

In [ ]:
rf_model_random = RandomForestRegressor(max_depth=rf_regressor.best_params_["max_depth"], 
                                 max_features=rf_regressor.best_params_["max_features"], 
                                 min_samples_split=rf_regressor.best_params_["min_samples_split"], 
                                 n_estimators=rf_regressor.best_params_["n_estimators"], 
                                 n_jobs=-1, oob_score=True)
rf_model_random.fit(train_inputs, train_targets)
train_preds = rf_model_random.predict(train_inputs)
val_preds = rf_model_random.predict(val_inputs)
print('Train r2 score: ', r2_score(train_preds, train_targets))
print('Test r2 score: ', r2_score(val_preds,val_targets ))
train_rmse_rf_random = np.sqrt(mean_squared_error(train_preds, train_targets))
test_rmse_rf_random = np.sqrt(mean_squared_error(val_targets, val_preds))
print(f'Train RMSE: {train_rmse_rf_random:n}')
print(f'Test RMSE: {test_rmse_rf_random:n}')

In [ ]:
# # Random Forest Model Feature Importance
rf_feature_importance = pd.Series(index = train_inputs.columns, data = np.abs(rf_model_random.feature_importances_))
n_features = (rf_feature_importance>0).sum()
print(f'{n_features} features with reduction of {(1-n_features/len(rf_feature_importance))*100:2.2f}%')
rf_feature_importance.sort_values().plot(kind = 'bar', figsize = (20,5));

## KNN Regressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

params_knn = {
    'n_neighbors' : [3, 5, 7, 9, 11],
}

knn_model = KNeighborsRegressor()
knn_regressor = GridSearchCV(knn_model, params_knn, scoring='neg_root_mean_squared_error', cv=10, n_jobs=-1)
knn_regressor.fit(train_inputs, train_targets)
print(f'Optimal neighbors: {knn_regressor.best_params_["n_neighbors"]}')
print(f'Best score: {knn_regressor.best_score_}')

In [ ]:
knn_model = KNeighborsRegressor(n_neighbors=knn_regressor.best_params_["n_neighbors"])
knn_model.fit(train_inputs, train_targets)
train_preds = knn_model.predict(train_inputs)
val_preds = knn_model.predict(val_inputs)
print('Train r2 score: ', r2_score(train_preds, train_targets))
print('Test r2 score: ', r2_score(val_targets, val_preds))
train_rmse_knn = np.sqrt(mean_squared_error(train_preds, train_targets))
test_rmse_knn = np.sqrt(mean_squared_error(val_targets, val_preds))
print(f'Train RMSE: {train_rmse_knn:n}')
print(f'Test RMSE: {test_rmse_knn:n}')

## Light GBM

In [ ]:

import lightgbm as lgb
from scipy.stats import uniform, randint
from sklearn.model_selection import RandomizedSearchCV

params = {
    "learning_rate": uniform(0.001,1),
    "n_estimators": randint(100,1000),
    "max_depth": randint(1,10),
    "colsample_bytree": uniform(0.1,0.8),
    "reg_alpha": [0.0001,0.001,0.01,0.1,1,10],
    "reg_lambda": [0.0001,0.001,0.01,0.1,1,10]
}

lgb_model = lgb.LGBMRegressor()
lgb_regressor = RandomizedSearchCV(lgb_model, params, cv=6, n_iter=100, scoring='neg_mean_squared_error', return_train_score=True, verbose=3, n_jobs=-1)
lgb_regressor.fit(train_inputs, train_targets)
print(f'Optimal lr: {lgb_regressor.best_params_["learning_rate"]}')
print(f'Optimal colsample_bytree: {lgb_regressor.best_params_["colsample_bytree"]}')
print(f'Optimal n_estimators: {lgb_regressor.best_params_["n_estimators"]}')
print(f'Best score: {lgb_regressor.best_score_}')

In [ ]:
lgb_model = lgb.LGBMRegressor(learning_rate=lgb_regressor.best_params_["learning_rate"], 
                         colsample_bytree=lgb_regressor.best_params_["colsample_bytree"], 
                         max_depth=4, alpha=3, subsample=.5, 
                         n_estimators=lgb_regressor.best_params_["n_estimators"], n_jobs=-1)
lgb_model.fit(train_inputs, train_targets)
train_preds = lgb_model.predict(train_inputs)
val_preds = lgb_model.predict(val_inputs)
print('Train r2 score: ', r2_score(train_preds, train_targets))
print('Test r2 score: ', r2_score(val_targets, val_preds))
train_rmse_lgb = np.sqrt(mean_squared_error(train_preds, train_targets))
test_rmse_lgb = np.sqrt(mean_squared_error(val_targets, val_preds))
print(f'Train RMSE XGB: {train_rmse_lgb:n}')
print(f'Test RMSE XGB: {test_rmse_lgb:n}')

In [ ]:
# XG Forest Model Feature Importance
rf_feature_importance = pd.Series(index = train_inputs.columns, data = np.abs(lgb_model.feature_importances_))
n_features = (rf_feature_importance>0).sum()
print(f'{n_features} features with reduction of {(1-n_features/len(rf_feature_importance))*100:2.2f}%')
rf_feature_importance.sort_values().plot(kind = 'bar', figsize = (20,5));

## Stacking Regressor

### Stack 1 - Ridge Lasso RF

In [ ]:
rf_model_en = RandomForestRegressor(max_depth=200, max_features=0.4, min_samples_leaf=3, 
                                 min_samples_split=6, n_estimators=30, n_jobs=-1, oob_score=True)
rf_model_en.fit(train_inputs, train_targets)
train_preds = rf_model_en.predict(train_inputs)
val_preds = rf_model_en.predict(val_inputs)
print('Train r2 score: ', r2_score(train_preds, train_targets))
print('Test r2 score: ', r2_score(val_targets, val_preds))
train_rmse_stack1 = np.sqrt(mean_squared_error(train_preds, train_targets))
test_rmse_stack1 = np.sqrt(mean_squared_error(val_targets, val_preds))
print(f'Train RMSE: {train_rmse_stack1:.4f}')
print(f'Test RMSE: {test_rmse_stack1:.4f}')

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
from matplotlib import pyplot

# # get a stacking ensemble of models
# def get_stacking():
#     # define the base models
#     base_models = list()
#     base_models.append(('ridge', ridge_model))
#     base_models.append(('lasso', lasso_model))
#     base_models.append(('rf', rf_model_en))
#     # define meta learner model
#     learner = LinearRegression()
#     # define the stacking ensemble
#     model = StackingRegressor(estimators=base_models, final_estimator=learner, cv=10)
#     return model
 
# # get a list of models to evaluate
# def get_models():
#     models = dict()
#     models['ridge'] = ridge_model
#     models['lasso'] = lasso_model
#     models['rf_en'] = rf_model_en
#     models['stacking'] = get_stacking()
#     return models
 
# # evaluate a given model using cross-validation
# def evaluate_model(model, X, y):
#     cv = RepeatedKFold(n_splits=10, n_repeats=5, random_state=19)
#     scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
#     return scores

# # get the models to evaluate
# models = get_models()
# # evaluate the models and store results
# results, names = list(), list()
# for name, model in models.items():
#     scores = evaluate_model(model, train_inputs, train_targets)
#     results.append(scores)
#     names.append(name)
#     print(f'{name} {mean(scores):.3f} {std(scores):.3f}')
# # plot model performance for comparison
# pyplot.boxplot(results, labels=names, showmeans=True)
# pyplot.show()

In [ ]:
# define the base models
base_models = list()
base_models.append(('ridge', ridge_model))
base_models.append(('lasso', lasso_model))
base_models.append(('xgb', xgb_model))
base_models.append(('lgm', lgb_model))
# define meta learner model
learner = LinearRegression()
# define the stacking ensemble
stack1 = StackingRegressor(estimators=base_models, final_estimator=learner, cv=10)
# fit the model on all available data
stack1.fit(train_inputs, train_targets)

In [ ]:
train_preds = stack1.predict(train_inputs)
val_preds = stack1.predict(val_inputs)
print('Train r2 score: ', r2_score(train_preds, train_targets))
print('Test r2 score: ', r2_score(val_targets, val_preds))
train_rmse_stack1 = np.sqrt(mean_squared_error(train_preds, train_targets))
test_rmse_stack1 = np.sqrt(mean_squared_error(val_targets, val_preds))
print(f'Train RMSE: {train_rmse_stack1:.4f}')
print(f'Test RMSE: {test_rmse_stack1:.4f}')

### Stack 2 - RF 1 / 2 / 3

In [ ]:
# define the base models
base_model = list()
base_model.append(('rf1', rf_model_random))
base_model.append(('rf2', rf_model_en))
base_model.append(('rf3', RandomForestRegressor(max_depth=8, max_features=0.1, min_samples_leaf=3, 
                                                min_samples_split=2, n_estimators=250, n_jobs=-1, oob_score=False)))
# define meta learner model
learner = LinearRegression()
# define the stacking ensemble
stack2 = StackingRegressor(estimators=base_model, final_estimator=learner, cv=10)
# fit the model on all available data
stack2.fit(train_inputs, train_targets)

In [ ]:
train_preds = stack2.predict(train_inputs)
val_preds = stack2.predict(val_inputs)
print('Train r2 score: ', r2_score(train_preds, train_targets))
print('Test r2 score: ', r2_score(val_targets, val_preds))
train_rmse_stack2 = np.sqrt(mean_squared_error(train_preds, train_targets))
test_rmse_stack2 = np.sqrt(mean_squared_error(val_targets, val_preds))
print(f'Train RMSE: {train_rmse_stack2:.4f}')
print(f'Test RMSE: {test_rmse_stack2:.4f}')

## Model Comparision

In [ ]:
from tabulate import tabulate 
#https://www.geeksforgeeks.org/display-the-pandas-dataframe-in-table-style/
dict = {'Models'     : ['Linear Regression','Ridge Linear Regression','Lasso','XGB','RF -Random','KNN','LightGBM','Stack1','Stack2' ],
        'Train RMSE' : [train_rmse_linreg,train_rmse_ridge,train_rmse_lasso,train_rmse_xgb,train_rmse_rf_random,train_rmse_knn,train_rmse_lgb,train_rmse_stack1,train_rmse_stack2],
        'Test RMSE'  : [test_rmse_linreg,test_rmse_ridge,test_rmse_lasso,test_rmse_xgb,test_rmse_rf_random,test_rmse_knn,test_rmse_lgb,test_rmse_stack1,test_rmse_stack2]  
        }
df = pd.DataFrame(dict)
print(tabulate(df, headers = 'keys', tablefmt = 'psql')) 

# Loading the Test Data and using the Test Data to Predict the target Variable

In [ ]:
df_test.head()

In [ ]:
## Using the model built on the Training set to predict on the Test Set
df_test_for_prediction = df_test.drop('RegistrationNumber',axis=1, inplace=False)
prediction_linreg = linreg_model.predict(df_test_for_prediction)
#prediction_rf = rf_model.predict(df_test_for_prediction)

#prediction_rf_random = rf_model_random.predict(df_test_for_prediction)

prediction_xgb_model = xgb_model.predict(df_test_for_prediction)
prediction_ridge_model = ridge_model.predict(df_test_for_prediction)
prediction_knn_model = knn_model.predict(df_test_for_prediction)

prediction_lasso_model = lasso_model.predict(df_test_for_prediction)
prediction_stack1_model = stack1.predict(df_test_for_prediction)
prediction_stack2_model = stack2.predict(df_test_for_prediction)

prediction_lgb_regressor = lgb_regressor.predict(df_test_for_prediction)


prediction_ridge_model

# Saving the output in a data frame and then exporting it to a '.csv' file with the appropriate 'Registration Number'.

In [ ]:
solution_df = pd.DataFrame(df_test['RegistrationNumber'])
solution_df

In [ ]:
#solution_df['Annual Turnover'] = prediction_rf

#solution_df['Annual Turnover'] = np.expm1(prediction_rf).astype('int64')

#solution_df['Annual Turnover'] = np.expm1(prediction_rf_random).astype('int64')


#solution_df['Annual Turnover'] = np.expm1(prediction_xgb_model).astype('int64')

#solution_df['Annual Turnover'] = (np.expm1(np.square(prediction_ridge_model)).astype('int64') )         #BEST

#solution_df['Annual Turnover'] = np.expm1(prediction_knn_model).astype('int64')

#solution_df['Annual Turnover'] = np.expm1(prediction_lasso_model).astype('int64')

#solution_df['Annual Turnover'] = np.expm1(prediction_stack1_model).astype('int64')

#solution_df['Annual Turnover'] = np.expm1(prediction_stack2_model).astype('int64')
#solution_df['Annual Turnover'] = np.expm1(prediction_stack2_model).astype('int64')


#solution_df['Annual Turnover'] = (np.expm1(np.square(prediction_lgb_regressor)).astype('int64') )   

solution_df['Annual Turnover'] = (np.expm1(prediction_stack2_model).astype('int64') )   


solution_df = solution_df.rename(columns={'RegistrationNumber': 'Registration Number'})
solution_df

In [ ]:
## Setting the directory to export the file as a '.csv'

import os
os.chdir(folderPath + '/output/')

In [ ]:
## Exporting the data frame to a '.csv' file and setting the index = False as we do want the index

solution_df.to_csv('Submission_stack2_log.csv',index=False)

# END